# Batch Process Inferences

In [8]:
from utils import VideoReaders, DetectorLoader
import numpy as np, glob, os
import matplotlib.pyplot as plt
import cv2
from skimage import segmentation

In [7]:
model = DetectorLoader.YOLACT('./weights/yolact_resnet50_54_800000.pth', threshold = 0.11)

/home/ubuntu/anaconda3/envs/YOLACT/lib/python3.7/site-packages/torch/jit/_recursive.py:235: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/home/ubuntu/anaconda3/envs/YOLACT/lib/python3.7/site-packages/torch/jit/_recursive.py:235: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/home/ubuntu/anaconda3/envs/YOLACT/lib/python3.7/site-packages/torch/jit/_recursive.py:235: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))


In [14]:
identities = sorted([os.path.splitext(os.path.basename(x))[0] for x in glob.glob('../datasets/slip-fall/no-fall-database/*.mp4')])

In [15]:
for video in identities:
    
    frame_provider = VideoReaders.VideoReader('../datasets/slip-fall/no-fall-database/{}.mp4'.format(video))
    video_length, shape = frame_provider.properties()
    
    output = []
    frames = []
    boundaries = []

    for frame in frame_provider:
        frames.append(frame)
        c, s, bb, ma = model.predict(frame)
        idx = np.where(c == 0)
        pixelwise_arrays = []
        for item in idx:
            for n, qq in enumerate(item):
                pixelwise = np.zeros_like(ma[qq,...])
                pixelwise = ma[qq,...].astype(np.uint8)
                pixelwise[np.where(pixelwise == 1)] = n + 1
                pixelwise_arrays.append(pixelwise)

        filtered = []
        for array in pixelwise_arrays:
            if np.max(array) != 0:
                filtered.append(array)

        stacked = np.sum(filtered, axis = 0)
        output.append(stacked)

    for im, mask in zip(frames, output):
        boundaries.append(segmentation.mark_boundaries(im, mask, mode = 'thick', color = (1,0,0)))


    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter("./results/no-slip-videos/{}.mp4".format(video), 0x7634706d, 5.0, max([x.shape for x in boundaries])[:-1][::-1])
    for capture in boundaries:
        out_capture = (capture * 255).astype(np.uint8)
        writer.write(out_capture)
    writer.release()
